<div align='center'>
<img src="https://data.seattle.gov/api/assets/82167BD8-353E-4232-96AD-72F5BF83F1D3?Official-Seattle-Logo-White__1_.png" alt="Seattle logo">  
</div>

---

<div align='center'> Projet 4 :Anticipez les besoins en consommation de bâtiments </div>
<div align='center'> Notebook: Modelisation de la prédiction de la consommation energétique
</div>


# MISSION

La ville de Seattle nous a confié une mission cruciale pour atteindre son objectif de ville neutre en émissions de carbone en 2050 : **prédire les émissions de CO² et la consommation d'énergie des bâtiments non résidentiels**, en évitant les recours à des relevés onéreux.   
En s'appuyant sur les données collectées avec rigueur en 2016, notre équipe s'est engagée dans la **conception d'un modèle prédictif** qui exploite les caractéristiques structurelles des bâtiments.    

Ce rapport détaille **les méthodes et les résultats** de notre étude sur la **prédiction de la consommation énergétique** des bâtiments. Après une analyse exploratoire approfondie, ce document se concentre sur l'évaluation de divers modèles prédictifs et explore l'influence de l'ENERGY STAR Score sur la précision de ces prédictions.

# PREPARATION DES DONNEES AU ML

In [1]:
# Importation des bibliothèques pour l'analyse de données
import numpy as np
import pandas as pd
import time

# Importation des bibliothèques pour la visualisation de données
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Importation des bibliothèques pour le prétraitement des données
from sklearn.preprocessing import FunctionTransformer, StandardScaler, RobustScaler
from sklearn.feature_selection import SelectFromModel

# Importation des modèles de machine learning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import (RandomForestRegressor, BaggingRegressor,AdaBoostRegressor, StackingRegressor)
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# Importation des outils pour la sélection de modèles et l'évaluation
from sklearn.model_selection import (
    train_test_split, cross_val_score, RandomizedSearchCV,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Importation des distributions statistiques pour la recherche aléatoire
from scipy.stats import randint, uniform, reciprocal, expon, loguniform

# Importation des outils pour la création de pipelines
from sklearn.pipeline import Pipeline, make_pipeline

In [2]:
pip install shap

In [ ]:
# Importation de SHAP pour la visualisation de données
import shap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Chargement du dataset
chemin_fichier = '/content/drive/MyDrive/Data/df_model_energy_filtred.csv'

# Chargement des données
df_model= pd.read_csv(chemin_fichier)
#Ajustement d'affichage
pd.set_option('display.max_columns', 50)

In [ ]:
df_model.describe()

In [ ]:
# Sélection des caractéristiques et définition de la variable cible pour la prédiction de la consommation énergétique
X = df_model.drop(['log_SiteEnergyUse(kBtu)','ENERGYSTARScore','log_TotalGHGEmissions'], axis=1)
y = df_model['log_SiteEnergyUse(kBtu)'].copy()

In [ ]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print('Train set :', X_train.shape)
print('Test set :', X_test.shape)

# REGRESSION LINEAIRE

In [ ]:
lin_reg = Pipeline([
    ('linear_regression', LinearRegression())
])

In [ ]:
# Entraînement du modèle
lr_start_time = time.time()
lin_reg.fit(X_train, y_train)
lr_end_time = time.time()

lin_reg_predict = lin_reg.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", lin_reg_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
lin_reg_rmse = mean_squared_error(y_train, lin_reg_predict, squared=False)
lin_reg_mae = mean_absolute_error(y_train, lin_reg_predict)
lin_reg_r2 = r2_score(y_train, lin_reg_predict)

print('Score RMSE:', lin_reg_rmse.round(2))
print('Score MAE:', lin_reg_mae.round(2))
print('Coefficient de détermination R²:', lin_reg_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - lin_reg_predict

plt.figure(figsize=(10, 6))
plt.scatter(lin_reg_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(lin_reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = lin_reg.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions = {
    'linear_regression__fit_intercept': [True, False]
}

random_search = RandomizedSearchCV(
    lin_reg,
    param_distributions=param_distributions,
    n_iter=2,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search.fit(X_train, y_train)

print(f'Meilleurs paramètres: {random_search.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_lr = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_lr = mean_absolute_error(y_test, y_test_pred)
test_r2_lr = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_lr}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_lr}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_lr}')

## Avec StandardScaler

In [ ]:
lin_reg_with_scaler = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())
])

In [ ]:
# Entraînement du modèle
lr_scaler_start_time = time.time()
lin_reg_with_scaler.fit(X_train, y_train)
lr_scaler_end_time = time.time()

lin_reg_predict_with_scaler = lin_reg_with_scaler.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", lin_reg_predict_with_scaler[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
lin_reg_scaler_rmse = mean_squared_error(y_train, lin_reg_predict_with_scaler, squared=False)
lin_reg_scaler_mae = mean_absolute_error(y_train, lin_reg_predict_with_scaler)
lin_reg_scaler_r2 = r2_score(y_train, lin_reg_predict_with_scaler)

print('Score RMSE:', lin_reg_scaler_rmse.round(2))
print('Score MAE:', lin_reg_scaler_mae.round(2))
print('Coefficient de détermination R²:', lin_reg_scaler_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - lin_reg_predict_with_scaler

plt.figure(figsize=(10, 6))
plt.scatter(lin_reg_predict_with_scaler, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(lin_reg_with_scaler, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred_scaler = lin_reg_with_scaler.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred_scaler, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred_scaler)
test_r2 = r2_score(y_test, y_test_pred_scaler)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions = {
    'linear_regression__fit_intercept': [True, False]
}

random_search = RandomizedSearchCV(
    lin_reg_with_scaler,
    param_distributions=param_distributions,
    n_iter=2,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search.fit(X_train, y_train)

print(f'Meilleurs paramètres: {random_search.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_lr_scaler = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_lr_scaler = mean_absolute_error(y_test, y_test_pred)
test_r2_lr_scaler = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_lr_scaler}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_lr_scaler}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_lr_scaler}')

# LASSO

In [ ]:
lasso_reg = Pipeline([
    ('Lasso', Lasso(random_state=42))
])

In [ ]:
# Entraînement du modèle
lasso_start_time = time.time()
lasso_reg.fit(X_train, y_train)
lasso_end_time = time.time()

lasso_reg_predict = lasso_reg.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", lasso_reg_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
lasso_reg_rmse = mean_squared_error(y_train, lasso_reg_predict, squared=False)
lasso_reg_mae = mean_absolute_error(y_train, lasso_reg_predict)
lasso_reg_r2 = r2_score(y_train, lasso_reg_predict)

print('Score RMSE:', lasso_reg_rmse.round(2))
print('Score MAE:', lasso_reg_mae.round(2))
print('Coefficient de détermination R²:', lasso_reg_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - lasso_reg_predict

plt.figure(figsize=(10, 6))
plt.scatter(lasso_reg_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(lasso_reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = lasso_reg.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_lasso = {
    'Lasso__alpha': uniform(0.001, 100),
    'Lasso__max_iter': randint(1000, 10000),
    'Lasso__fit_intercept': [True, False]
}

n_iter_search = 10

random_search_lasso = RandomizedSearchCV(
    lasso_reg,
    param_distributions=param_distributions_lasso,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_lasso.fit(X_train, y_train)

print(f'Meilleurs paramètres pour Lasso: {random_search_lasso.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_lasso.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_lasso = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_lasso = mean_absolute_error(y_test, y_test_pred)
test_r2_lasso = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_lasso}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_lasso}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_lasso}')

## Avec StandardScaler

In [ ]:
lasso_reg_with_scaler = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', Lasso(random_state=42))
])

In [ ]:
# Entraînement du modèle
lasso_scaler_start_time = time.time()
lasso_reg_with_scaler.fit(X_train, y_train)
lasso_scaler_end_time = time.time()

lasso_reg_predict_with_scaler = lasso_reg_with_scaler.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", lasso_reg_predict_with_scaler[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
lasso_reg_scaler_rmse = mean_squared_error(y_train, lasso_reg_predict, squared=False)
lasso_reg_scaler_mae = mean_absolute_error(y_train, lasso_reg_predict)
lasso_reg_scaler_r2 = r2_score(y_train, lasso_reg_predict)

print('Score RMSE:', lasso_reg_scaler_rmse.round(2))
print('Score MAE:', lasso_reg_scaler_mae.round(2))
print('Coefficient de détermination R²:', lasso_reg_scaler_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - lasso_reg_predict_with_scaler

plt.figure(figsize=(10, 6))
plt.scatter(lasso_reg_predict_with_scaler, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(lasso_reg_with_scaler, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Utilisation de la validation croisée pour évaluer la performance du modèle
cv_scores = cross_val_score(lasso_reg_with_scaler, X_train, y_train, cv=10, scoring='neg_mean_squared_error')

# Calcul de la RMSE pour chaque fold
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred_scaler = lasso_reg_with_scaler.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred_scaler, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred_scaler)
test_r2 = r2_score(y_test, y_test_pred_scaler)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_lasso = {
    'lasso__alpha': uniform(0.001, 100),
    'lasso__max_iter': randint(1000, 10000),
    'lasso__fit_intercept': [True, False]
}

n_iter_search = 10

random_search_lasso = RandomizedSearchCV(
    lasso_reg_with_scaler,
    param_distributions=param_distributions_lasso,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_lasso.fit(X_train, y_train)

print(f'Meilleurs paramètres pour Lasso: {random_search_lasso.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_lasso.best_estimator_
y_test_pred_scaler = best_model.predict(X_test)

test_rmse_scaler_lasso = mean_squared_error(y_test, y_test_pred_scaler, squared=False)
test_mae_scaler_lasso = mean_absolute_error(y_test, y_test_pred_scaler)
test_r2_scaler_lasso = r2_score(y_test, y_test_pred_scaler)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_scaler_lasso}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_scaler_lasso}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_scaler_lasso}')

# RIDGE

In [ ]:
ridge_reg = Pipeline(steps=[
    ('ridge_regressor', Ridge(random_state=42))
])

In [ ]:
# Entraînement du modèle
ridge_start_time = time.time()
ridge_reg.fit(X_train, y_train)
ridge_end_time = time.time()

ridge_reg_predict = ridge_reg.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", ridge_reg_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
ridge_reg_rmse = mean_squared_error(y_train, ridge_reg_predict, squared=False)
ridge_reg_mae = mean_absolute_error(y_train, ridge_reg_predict)
ridge_reg_r2 = r2_score(y_train, ridge_reg_predict)

print('Score RMSE:', ridge_reg_rmse.round(2))
print('Score MAE:', ridge_reg_mae.round(2))
print('Coefficient de détermination R²:', ridge_reg_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - ridge_reg_predict

plt.figure(figsize=(10, 6))
plt.scatter(ridge_reg_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(ridge_reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = ridge_reg.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_ridge = {
    'ridge_regressor__alpha': loguniform(0.001, 100),
    'ridge_regressor__fit_intercept': [True, False],
    'ridge_regressor__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    'ridge_regressor__max_iter': [10000, 20000]
}

n_iter_search = 10

random_search_ridge = RandomizedSearchCV(
    ridge_reg,
    param_distributions=param_distributions_ridge,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_ridge.fit(X_train, y_train)

print(f'Meilleurs paramètres pour Ridge: {random_search_ridge.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_ridge.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_ridge = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_ridge = mean_absolute_error(y_test, y_test_pred)
test_r2_ridge = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_ridge}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_ridge}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_ridge}')

## Avec StandardScaler

In [ ]:
ridge_reg_with_scaler = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('ridge_regressor', Ridge())
])

In [ ]:
# Entraînement du modèle
ridge_scaler_start_time = time.time()
ridge_reg_with_scaler.fit(X_train, y_train)
ridge_scaler_end_time = time.time()

ridge_reg_predict_with_scaler = ridge_reg_with_scaler.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", ridge_reg_predict_with_scaler[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
ridge_reg_scaler_rmse = mean_squared_error(y_train, ridge_reg_predict_with_scaler, squared=False)
ridge_reg_scaler_mae = mean_absolute_error(y_train, ridge_reg_predict_with_scaler)
ridge_reg_scaler_r2 = r2_score(y_train, ridge_reg_predict_with_scaler)

print('Score RMSE:', ridge_reg_scaler_rmse.round(2))
print('Score MAE:', ridge_reg_scaler_mae.round(2))
print('Coefficient de détermination R²:', ridge_reg_scaler_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - ridge_reg_predict_with_scaler

plt.figure(figsize=(10, 6))
plt.scatter(ridge_reg_predict_with_scaler, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(ridge_reg_with_scaler, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred_scaler = ridge_reg_with_scaler.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred_scaler, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred_scaler)
test_r2 = r2_score(y_test, y_test_pred_scaler)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_ridge = {
    'ridge_regressor__alpha': loguniform(0.001, 100),
    'ridge_regressor__fit_intercept': [True, False],
    'ridge_regressor__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
    'ridge_regressor__max_iter': [10000, 20000]
}

n_iter_search = 10

random_search_ridge = RandomizedSearchCV(
    ridge_reg_with_scaler,
    param_distributions=param_distributions_ridge,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_ridge.fit(X_train, y_train)

print(f'Meilleurs paramètres pour Ridge: {random_search_ridge.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_ridge.best_estimator_
y_test_pred_scaler = best_model.predict(X_test)

test_rmse_scaler_ridge = mean_squared_error(y_test, y_test_pred_scaler, squared=False)
test_mae_scaler_ridge = mean_absolute_error(y_test, y_test_pred_scaler)
test_r2_scaler_ridge = r2_score(y_test, y_test_pred_scaler)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_scaler_ridge}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_scaler_ridge}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_scaler_ridge}')

#ELASTIC NET

In [ ]:
elastic_net_reg = Pipeline(steps=[
    (('elastic_net', ElasticNet(max_iter=10000, random_state=42)))
])

In [ ]:
# Entraînement du modèle
elastic_net_start_time = time.time()
elastic_net_reg.fit(X_train, y_train)
elastic_net_end_time = time.time()

elastic_net_reg_predict = elastic_net_reg.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", elastic_net_reg_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
elastic_net_reg_rmse = mean_squared_error(y_train, elastic_net_reg_predict, squared=False)
elastic_net_reg_mae = mean_absolute_error(y_train, elastic_net_reg_predict)
elastic_net_reg_r2 = r2_score(y_train, elastic_net_reg_predict)

print('Score RMSE:', elastic_net_reg_rmse.round(2))
print('Score MAE:', elastic_net_reg_mae.round(2))
print('Coefficient de détermination R²:', elastic_net_reg_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - elastic_net_reg_predict

plt.figure(figsize=(10, 6))
plt.scatter(elastic_net_reg_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(elastic_net_reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = elastic_net_reg.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_elastic_net = {
    'elastic_net__alpha': loguniform(0.1, 10000),
    'elastic_net__l1_ratio': loguniform(0.01, 1),
    'elastic_net__fit_intercept': [True, False]
}
n_iter_search = 10

random_search_elastic_net = RandomizedSearchCV(
    elastic_net_reg,
    param_distributions=param_distributions_elastic_net,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_elastic_net.fit(X_train, y_train)

print(f'Meilleurs paramètres pour Elastic Net: {random_search_elastic_net.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_elastic_net.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_elastic_net = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_elastic_net = mean_absolute_error(y_test, y_test_pred)
test_r2_elastic_net = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_elastic_net}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_elastic_net}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_elastic_net}')

## Avec StandardScaler

In [ ]:
elastic_net_reg_with_scaler = Pipeline(steps=[
    ('scaler', StandardScaler()),
    (('elastic_net', ElasticNet(max_iter=100, random_state=42)))
])

In [ ]:
# Entraînement du modèle
elastic_net_scaler_start_time = time.time()
elastic_net_reg_with_scaler.fit(X_train, y_train)
elastic_net_scaler_end_time = time.time()

elastic_net_reg_predict_with_scaler = elastic_net_reg_with_scaler.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", elastic_net_reg_predict_with_scaler[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
elastic_net_reg_scaler_rmse = mean_squared_error(y_train, elastic_net_reg_predict_with_scaler, squared=False)
elastic_net_reg_scaler__mae = mean_absolute_error(y_train, elastic_net_reg_predict_with_scaler)
elastic_net_reg_scaler_r2 = r2_score(y_train, elastic_net_reg_predict_with_scaler)

print('Score RMSE:', elastic_net_reg_scaler_rmse.round(2))
print('Score MAE:', elastic_net_reg_scaler__mae.round(2))
print('Coefficient de détermination R²:', elastic_net_reg_scaler_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - elastic_net_reg_predict_with_scaler

plt.figure(figsize=(10, 6))
plt.scatter(elastic_net_reg_predict_with_scaler, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(elastic_net_reg_with_scaler, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred_scaler = elastic_net_reg_with_scaler.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred_scaler, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred_scaler)
test_r2 = r2_score(y_test, y_test_pred_scaler)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_elastic_net = {
    'elastic_net__alpha': loguniform(0.1, 100),
    'elastic_net__l1_ratio': loguniform(0.1, 1),
    'elastic_net__fit_intercept': [True, False]
}

n_iter_search = 10

random_search_elastic_net = RandomizedSearchCV(
    elastic_net_reg_with_scaler,
    param_distributions=param_distributions_elastic_net,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_elastic_net.fit(X_train, y_train)

print(f'Meilleurs paramètres pour Elastic Net: {random_search_elastic_net.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_elastic_net.best_estimator_
y_test_pred_scaler = best_model.predict(X_test)

test_rmse_scaler_elastic_net = mean_squared_error(y_test, y_test_pred_scaler, squared=False)
test_mae_scaler_elastic_net = mean_absolute_error(y_test, y_test_pred_scaler)
test_r2_scaler_elastic_net = r2_score(y_test, y_test_pred_scaler)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_scaler_elastic_net}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_scaler_elastic_net}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_scaler_elastic_net}')

# BAGGING : RANDOM FOREST REGRESSOR

In [ ]:
bagging_rf = Pipeline(steps=[
    ('bagging_random_forest', BaggingRegressor(
        estimator=RandomForestRegressor(n_estimators=100, random_state=42),
        random_state=42
    ))
])

In [ ]:
# Entraînement du modèle
bagging_rf_start_time = time.time()
bagging_rf.fit(X_train, y_train)
bagging_rf_end_time = time.time()

bagging_rf_predict = bagging_rf.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", bagging_rf_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
bagging_rf_rmse = mean_squared_error(y_train, bagging_rf_predict, squared=False)
bagging_rf_mae = mean_absolute_error(y_train, bagging_rf_predict)
bagging_rf_r2 = r2_score(y_train, bagging_rf_predict)

print('Score RMSE:', bagging_rf_rmse.round(2))
print('Score MAE:', bagging_rf_mae.round(2))
print('Coefficient de détermination R²:', bagging_rf_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - bagging_rf_predict

plt.figure(figsize=(10, 6))
plt.scatter(bagging_rf_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(bagging_rf, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = bagging_rf.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_bagging_rf = {
    'bagging_random_forest__n_estimators': randint(10, 101),
    'bagging_random_forest__estimator__n_estimators': randint(10, 101),
    'bagging_random_forest__max_samples': uniform(0.5, 0.5),
    'bagging_random_forest__max_features': uniform(0.5, 0.5)

}

n_iter_search = 10

random_search_bagging_rf = RandomizedSearchCV(
    bagging_rf,
    param_distributions=param_distributions_bagging_rf,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_bagging_rf.fit(X_train, y_train)

print(f'Meilleurs paramètres pour Bagging RandomForestRegressor: {random_search_bagging_rf.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_bagging_rf.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_bagging = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_bagging = mean_absolute_error(y_test, y_test_pred)
test_r2_bagging = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_bagging}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_bagging}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_bagging}')

# BOOSTING : ADABOOST REGRESSOR

In [ ]:
ada_boost = Pipeline(steps=[
    ('ada_boost', AdaBoostRegressor(
        estimator=DecisionTreeRegressor(random_state=42),
        random_state=42
    ))
])

In [ ]:
# Entraînement du modèle
ada_boost_start_time = time.time()
ada_boost.fit(X_train, y_train)
ada_boost_end_time = time.time()

ada_boost_predict = ada_boost.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", ada_boost_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
ada_boost_rmse = mean_squared_error(y_train, ada_boost_predict, squared=False)
ada_boost_mae = mean_absolute_error(y_train, ada_boost_predict)
ada_boost_r2 = r2_score(y_train, ada_boost_predict)

print('Score RMSE:', ada_boost_rmse.round(2))
print('Score MAE:', ada_boost_mae.round(2))
print('Coefficient de détermination R²:', ada_boost_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - ada_boost_predict

plt.figure(figsize=(10, 6))
plt.scatter(ada_boost_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(ada_boost, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = ada_boost.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_ada_boost = {
    'ada_boost__n_estimators': randint(50, 500),
    'ada_boost__learning_rate': uniform(0.01, 1),
    'ada_boost__estimator__max_depth': randint(1, 11),
}

n_iter_search = 10

random_search_ada_boost = RandomizedSearchCV(
    ada_boost,
    param_distributions=param_distributions_ada_boost,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_ada_boost.fit(X_train, y_train)

print(f'Meilleurs paramètres pour AdaBoostRegressor: {random_search_ada_boost.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_ada_boost.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_boosting = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_boosting = mean_absolute_error(y_test, y_test_pred)
test_r2_boosting = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_boosting}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_boosting}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_boosting}')

# BOOSTING : XGBOOST REGRESSOR

In [ ]:
xgboost_reg = Pipeline([
    ('xgb', XGBRegressor(random_state=42))
])

In [ ]:
# Entraînement du modèle
xgboost_start_time = time.time()
xgboost_reg.fit(X_train, y_train)
xgboost_end_time = time.time()

xgboost_predict = xgboost_reg.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", xgboost_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
xgboost_rmse = mean_squared_error(y_train, ada_boost_predict, squared=False)
xgboost_mae = mean_absolute_error(y_train, ada_boost_predict)
xgboost_r2 = r2_score(y_train, ada_boost_predict)

print('Score RMSE:', xgboost_rmse.round(2))
print('Score MAE:', xgboost_mae.round(2))
print('Coefficient de détermination R²:', xgboost_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - xgboost_predict

plt.figure(figsize=(10, 6))
plt.scatter(xgboost_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(xgboost_reg, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = xgboost_reg.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_xgb = {
    'xgb__n_estimators': randint(50, 500),
    'xgb__learning_rate': uniform(0.01, 0.99),
    'xgb__max_depth': randint(3, 10),
    'xgb__min_child_weight': randint(1, 10),
    'xgb__subsample': uniform(0.5, 0.5),
    'xgb__colsample_bytree': uniform(0.5, 0.5)
}

n_iter_search = 10

random_search_xgb = RandomizedSearchCV(
    xgboost_reg,
    param_distributions=param_distributions_xgb,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_xgb.fit(X_train, y_train)

print(f'Meilleurs paramètres pour XGBRegressor: {random_search_xgb.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_xgb.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_boosting_xgb = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_boosting_xgb = mean_absolute_error(y_test, y_test_pred)
test_r2_boosting_xgb = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_boosting_xgb}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_boosting_xgb}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_boosting_xgb}')

# SVR

In [ ]:
svr = Pipeline([
 ('svr', SVR(kernel='rbf'))
])

In [ ]:
# Entraînement du modèle
svr_start_time = time.time()
svr.fit(X_train, y_train)
svr_end_time = time.time()

svr_predict = svr.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", svr_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
svr_rmse = mean_squared_error(y_train, ada_boost_predict, squared=False)
svr_mae = mean_absolute_error(y_train, ada_boost_predict)
svr_r2 = r2_score(y_train, ada_boost_predict)

print('Score RMSE:', svr_rmse.round(2))
print('Score MAE:', svr_mae.round(2))
print('Coefficient de détermination R²:', svr_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - svr_predict

plt.figure(figsize=(10, 6))
plt.scatter(svr_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(svr, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = svr.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_svr = {
    'svr__C': reciprocal(1e-4, 1e4),
    'svr__epsilon': expon(scale=0.1),
    'svr__gamma': expon(scale=0.1)
}

n_iter_search = 10

random_search_svr = RandomizedSearchCV(
    svr,
    param_distributions=param_distributions_svr,
    n_iter=n_iter_search,
    cv=5,
    scoring='r2',
    refit=True,
    random_state=42
)

random_search_svr.fit(X_train, y_train)

print(f'Meilleurs paramètres pour SVR: {random_search_svr.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_svr.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_svr = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_svr = mean_absolute_error(y_test, y_test_pred)
test_r2_svr = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_svr}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_svr}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_svr}')

## Avec StardardScaler

In [ ]:
svr_scaler = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(kernel='rbf'))
])

In [ ]:
# Entraînement du modèle
svr_scaler_start_time = time.time()
svr_scaler.fit(X_train, y_train)
svr_scaler_end_time = time.time()

svr_scaler_predict = svr_scaler.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", svr_scaler_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
svr_scaler_rmse = mean_squared_error(y_train, svr_scaler_predict, squared=False)
svr_scaler_mae = mean_absolute_error(y_train, svr_scaler_predict)
svr_scaler_r2 = r2_score(y_train, svr_scaler_predict)

print('Score RMSE:', svr_scaler_rmse.round(2))
print('Score MAE:', svr_scaler_mae.round(2))
print('Coefficient de détermination R²:', svr_scaler_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - svr_scaler_predict

plt.figure(figsize=(10, 6))
plt.scatter(svr_scaler_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(svr_scaler, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = svr_scaler.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions_svr = {
    'svr__C': reciprocal(1e-4, 1e4),
    'svr__epsilon': expon(scale=0.1),
    'svr__gamma': expon(scale=0.1)
}

n_iter_search = 10

random_search_svr_scaler = RandomizedSearchCV(
    svr_scaler,
    param_distributions=param_distributions_svr,
    n_iter=n_iter_search,
    cv=5,
    scoring='neg_mean_absolute_error',
    refit=True,
    random_state=42
)

random_search_svr_scaler.fit(X_train, y_train)

print(f'Meilleurs paramètres pour SVR: {random_search_svr_scaler.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_svr_scaler.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_svr_scaler = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_svr_scaler = mean_absolute_error(y_test, y_test_pred)
test_r2_svr_scaler = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_svr_scaler}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_svr_scaler}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_svr_scaler}')

# STACKING :  STACKING REGRESSOR

In [ ]:
# Définition des modèles de base pour le stacking
linear_regression = LinearRegression()
svr = make_pipeline(StandardScaler(), SVR())
ridge = Ridge()

estimators = [
    ('linear_regression', linear_regression),
    ('svr', svr),
    ('ridge', ridge)
]

meta_model = LinearRegression()

stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_model
)

In [ ]:
# Entraînement du modèle
stacking_regressor_start_time = time.time()
stacking_regressor.fit(X_train, y_train)
stacking_regressor_end_time = time.time()

stacking_regressor_predict = stacking_regressor.predict(X_train)

In [ ]:
# Affichage des prédictions et des valeurs réelles
print("Prédictions des 5 premières instances:", stacking_regressor_predict[:5].round(2))
print("Valeurs réelles des 5 premières instances:", y_train.iloc[:5].values.round(2))

In [ ]:
# Calcul des métriques d'évaluation
stacking_regressor_rmse = mean_squared_error(y_train, stacking_regressor_predict, squared=False)
stacking_regressor_mae = mean_absolute_error(y_train, stacking_regressor_predict)
stacking_regressor_r2 = r2_score(y_train, stacking_regressor_predict)

print('Score RMSE:', stacking_regressor_rmse.round(2))
print('Score MAE:', stacking_regressor_mae.round(2))
print('Coefficient de détermination R²:', stacking_regressor_r2.round(2))

In [ ]:
# Graphique des résidus
residuals = y_train - stacking_regressor_predict

plt.figure(figsize=(10, 6))
plt.scatter(stacking_regressor_predict, residuals, alpha=0.5)
plt.title('Graphique des résidus')
plt.xlabel('Valeurs prédites')
plt.ylabel('Résidus')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Validation croisée
cv_scores = cross_val_score(stacking_regressor, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
cv_rmse_scores = np.sqrt(-cv_scores)

print("Scores de la validation croisée (RMSE):", cv_rmse_scores)
print("Moyenne des scores de la validation croisée (RMSE):", cv_rmse_scores.mean())

In [ ]:
# Prédiction sur l'ensemble de test
y_test_pred = stacking_regressor.predict(X_test)

test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test: {test_rmse}')
print(f'Score MAE sur l\'ensemble de test: {test_mae}')
print(f'Coefficient de détermination R² sur l\'ensemble de test: {test_r2}')

In [ ]:
# Recherche des meilleurs hyperparamètres
param_distributions = {
    'linear_regression__fit_intercept': [True, False],
    'svr__svr__C': uniform(1, 10),
    'svr__svr__gamma': ['scale', 'auto'],
    'ridge__alpha': uniform(0.1, 10),
}

n_iter_search = 10

random_search_stacking = RandomizedSearchCV(
    estimator=stacking_regressor,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='r2',
    random_state=42
)

random_search_stacking.fit(X_train, y_train)

print(f'Meilleurs paramètres pour Stacking: {random_search_stacking.best_params_}')

In [ ]:
# Évaluation des Performances du Meilleur Modèle sur l'Ensemble de Test
best_model = random_search_stacking.best_estimator_
y_test_pred = best_model.predict(X_test)

test_rmse_stacking = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae_stacking = mean_absolute_error(y_test, y_test_pred)
test_r2_stacking = r2_score(y_test, y_test_pred)

print(f'Score RMSE sur l\'ensemble de test avec le meilleur modèle: {test_rmse_stacking}')
print(f'Score MAE sur l\'ensemble de test avec le meilleur modèle: {test_mae_stacking}')
print(f'Coefficient de détermination R² sur l\'ensemble de test avec le meilleur modèle: {test_r2_stacking}')

# ANALYSE DES RESULTATS

In [ ]:
# Calcul des temps d'exécution pour différents modèles de régression
lr_execution_time = lr_end_time - lr_start_time
lr_scaler_execution_time = lr_scaler_end_time - lr_scaler_start_time
lasso_execution_time = lasso_end_time - lasso_start_time
lasso_scaler_execution_time = lasso_scaler_end_time - lasso_scaler_start_time
ridge_execution_time = ridge_end_time - ridge_start_time
ridge_scaler_execution_time = ridge_scaler_end_time - ridge_scaler_start_time
elastic_net_execution_time = elastic_net_end_time - elastic_net_start_time
elastic_net_scaler_execution_time = elastic_net_scaler_end_time - elastic_net_scaler_start_time
bagging_rf_execution_time = bagging_rf_end_time - bagging_rf_start_time
ada_boost_execution_time = ada_boost_end_time - ada_boost_start_time
xgboost_executive_time = xgboost_end_time - xgboost_start_time
svr_executive_time = svr_end_time - svr_start_time
svr_scaler_executive_time = svr_scaler_end_time - svr_scaler_start_time
stacking_regressor_execution_time = stacking_regressor_end_time - stacking_regressor_start_time

In [ ]:
# Creation & Affichage du tableau de résultats
results = {
    'Model': [
        'Linear Regression',
        'Linear Regression avec StandardScaler',
        'Lasso',
        'Lasso avec StandardScaler',
        'Ridge',
        'Ridge avec StandardScaler',
        'Elastic Net',
        'Elastic Net avec StandardScaler',
        'Bagging RandomForest',
        'Boosting AdaBoost',
        'Boosting XGBoost',
        'SVR',
        'SVR avec StandardScaler',
        'Stacking Regressor'
    ],
    'MAE': [
        test_mae_lr,
        test_mae_lr_scaler,
        test_mae_lasso,
        test_mae_scaler_lasso,
        test_mae_ridge,
        test_mae_scaler_ridge,
        test_mae_elastic_net,
        test_mae_scaler_elastic_net,
        test_mae_bagging,
        test_mae_boosting,
        test_mae_boosting_xgb,
        test_mae_svr,
        test_mae_svr_scaler,
        test_mae_stacking
    ],
    'RMSE': [
        test_rmse_lr,
        test_rmse_lr_scaler,
        test_rmse_lasso,
        test_rmse_scaler_lasso,
        test_rmse_ridge,
        test_rmse_scaler_ridge,
        test_rmse_elastic_net,
        test_rmse_scaler_elastic_net,
        test_rmse_bagging,
        test_rmse_boosting,
        test_rmse_boosting_xgb,
        test_rmse_svr,
        test_rmse_svr_scaler,
        test_rmse_stacking
    ],
    'R2': [
        test_r2_lr,
        test_r2_lr_scaler,
        test_r2_lasso,
        test_r2_scaler_lasso,
        test_r2_ridge,
        test_r2_scaler_ridge,
        test_r2_elastic_net,
        test_r2_scaler_elastic_net,
        test_r2_bagging,
        test_r2_boosting,
        test_r2_boosting_xgb,
        test_r2_svr,
        test_r2_svr_scaler,
        test_r2_stacking
    ],
    'Execution Time (s)': [
        lr_execution_time,
        lr_scaler_execution_time,
        lasso_execution_time,
        lasso_scaler_execution_time,
        ridge_execution_time,
        ridge_scaler_execution_time,
        elastic_net_execution_time,
        elastic_net_scaler_execution_time,
        bagging_rf_execution_time,
        ada_boost_execution_time,
        xgboost_executive_time,
        svr_executive_time,
        svr_scaler_executive_time,
        stacking_regressor_execution_time
    ]
}


results_df = pd.DataFrame(results)

results_df

**En conclusion :**

**Régression Linéaire:** Performances solides avec et sans standardisation, mais légèrement améliorées avec la standardisation.  

**Lasso:** Moins performant que la régression linéaire, avec une détérioration notable lorsqu'il est combiné avec la standardisation.
Ridge: Très compétitif, avec une amélioration notable des performances lorsqu'il est combiné avec la standardisation, ce qui en fait le modèle de choix.  

**Elastic Net:** Performances intermédiaires, avec une légère amélioration grâce à la standardisation.  

**Bagging avec RandomForest:** Bonnes performances mais temps d'exécution nettement plus long, ce qui le rend moins attrayant pour des applications nécessitant une réponse rapide.  

**AdaBoost:** Performances comparables au Bagging avec RandomForest mais avec un temps d'exécution beaucoup plus court.  

**XGBoost:** Performances compétitives, mais avec un temps d'exécution plus long que les modèles linéaires.  

**SVR:** Performances améliorées avec la standardisation, mais reste moins performant que le modèle Ridge standardisé.  

**Stacking Regressor:** Bonnes performances mais avec le temps d'exécution le plus long, ce qui peut être un inconvénient.  

**Le modèle Ridge avec StandardScaler se distingue comme le meilleur choix** en raison de son excellent équilibre entre précision (avec le MAE le plus bas et le score R2 le plus élevé parmi les modèles testés) et rapidité d'exécution, ce qui le rend idéal pour une utilisation pratique.

## Feature Importances

In [ ]:
# Extraction des noms des caractéristiques
feature_names = X_train.columns.tolist()

In [ ]:
# Affichage des importances des caractéristiques pour le modèle Ridge
ridge_estimator = ridge_reg_with_scaler.named_steps['ridge_regressor']
coefficients = ridge_estimator.coef_

importances = np.abs(coefficients)

indices = np.argsort(importances)[::-1]

print("Feature importances:")
for f in range(X_train.shape[1]):
    print(f"{f + 1}. feature {feature_names[indices[f]]} ({importances[indices[f]]})")

In [ ]:
# Création du graphique
plt.figure(figsize=(12, 6))
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices], color="r", align="center")
plt.xticks(range(X_train.shape[1]), [feature_names[i] for i in indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
# Création d'un graphique en cascade SHAP pour l'instance de test sélectionnée avec le modèle Ridge
explainer = shap.Explainer(ridge_reg_with_scaler.predict, X_train)

i = 10
shap_values = explainer(X_test.iloc[i:i+1])


shap.plots.waterfall(shap_values[0])

In [ ]:
# Génération du résumé SHAP pour l'ensemble de test
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)

**En conclusion :**
 L'analyse des coefficients du modèle Ridge et l'interprétation SHAP ont fourni des insights précieux sur les facteurs influençant la consommation énergétique des propriétés.  

La **surface totale de la propriété** s'est révélée être le **facteur le plus déterminant**, indiquant une corrélation directe avec la consommation d'énergie. Les types de propriété, tels que les espaces de stockage et de vente au détail, ainsi que des facteurs géographiques comme la latitude, ont également montré une influence notable.

**L'approche SHAP** a permis de mettre en lumière l'impact spécifique de chaque caractéristique sur les prédictions individuelles, révélant des interactions complexes entre les variables. Par exemple, une plus grande surface totale a été associée à une consommation d'énergie réduite dans le cas spécifique analysé, tandis que le type de propriété et la latitude ont eu un impact positif sur la consommation d'énergie.

Ces informations sont **cruciales pour la prise de décision** en matière d'efficacité énergétique et pourront guider les efforts d'amélioration des performances énergétiques des bâtiments. En intégrant ces connaissances dans la planification et la gestion des propriétés, il est possible de cibler des interventions plus efficaces pour réduire la consommation d'énergie et les émissions de gaz à effet de serre.

**Le modèle Ridge, enrichi par l'analyse SHAP**, offre donc un outil puissant pour comprendre et prédire la consommation énergétique, et par extension, pour contribuer à la création d'un environnement bâti plus durable.

# ENERGYSTAR Score

In [ ]:
X = X.join(df_model['ENERGYSTARScore'])

In [ ]:
X= X.dropna()

In [ ]:
# Préparation des données avec et sans la caractéristique 'ENERGYSTARScore
best_params = random_search_ridge.best_params_

X_with_feature = X.copy()
X_without_feature = X.drop('ENERGYSTARScore', axis=1)

y = y.loc[X_with_feature.index]
X_without_feature = X_without_feature.loc[y.index]

In [ ]:
# Division des données en ensembles d'entraînement et de test avec et sans la caractéristique 'ENERGYSTARScore' et nettoyage des meilleurs paramètres
X_train_with, X_test_with, y_train, y_test = train_test_split(X_with_feature, y, test_size=0.2, random_state=42)

X_train_without = X_without_feature.loc[X_train_with.index]
X_test_without = X_without_feature.loc[X_test_with.index]

cleaned_params = {param_key.split('__')[-1]: param_value for param_key, param_value in best_params.items()}

In [ ]:
# Entraînement du modèle Ridge avec la caractéristique 'ENERGYSTARScore'
model_with_feature = Ridge(**cleaned_params)
model_with_feature.fit(X_train_with, y_train)

# Entraînement du modèle Ridge sans la caractéristique 'ENERGYSTARScore'
model_without_feature = Ridge(**cleaned_params)
model_without_feature.fit(X_train_without, y_train)

In [ ]:
# Évaluation des modèles à l'aide de la validation croisée
score_with_feature = -cross_val_score(model_with_feature, X_train_with, y_train, cv=5, scoring='neg_mean_absolute_error').mean()
score_without_feature = -cross_val_score(model_without_feature, X_train_without, y_train, cv=5, scoring='neg_mean_absolute_error').mean()

print(f"Score avec la caractéristique 'ENERGYSTARScore' (MAE): {score_with_feature}")
print(f"Score sans la caractéristique 'ENERGYSTARScore' (MAE): {score_without_feature}")

In [ ]:
# Prédictions sur l'ensemble de test avec la caractéristique 'ENERGYSTARScore'
y_pred_with = model_with_feature.predict(X_test_with)

# Prédictions sur l'ensemble de test sans la caractéristique 'ENERGYSTARScore'
y_pred_without = model_without_feature.predict(X_test_without)

In [ ]:
# Calcul du RMSE & R² pour le modèle avec ou sans la caractéristique 'ENERGYSTARScore'
rmse_with_feature = np.sqrt(mean_squared_error(y_test, y_pred_with))
rmse_without_feature = np.sqrt(mean_squared_error(y_test, y_pred_without))

r2_with_feature = r2_score(y_test, y_pred_with)
r2_without_feature = r2_score(y_test, y_pred_without)

print(f"RMSE avec la caractéristique 'ENERGYSTARScore': {rmse_with_feature}")
print(f"RMSE sans la caractéristique 'ENERGYSTARScore': {rmse_without_feature}")
print(f"R² avec la caractéristique 'ENERGYSTARScore': {r2_with_feature}")
print(f"R² sans la caractéristique 'ENERGYSTARScore': {r2_without_feature}")

In [ ]:
# Visualisation des résultats pour comparer les scores MAE
labels = ['Avec ENERGYSTARScore', 'Sans ENERGYSTARScore']
scores = [score_with_feature, score_without_feature]

plt.bar(labels, scores, color=['blue', 'green'])
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Comparaison des performances du modèle Ridge')
plt.show()

In [ ]:
# Tracé des prédictions avec la caractéristique 'ENERGYSTARScore'
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_with, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--k')
plt.title("Prédictions avec 'ENERGYSTARScore'")
plt.xlabel('Valeurs réelles')
plt.ylabel('Prédictions')

# Tracé des prédictions sans la caractéristique 'ENERGYSTARScore'
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred_without, alpha=0.3, color='g')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--k')
plt.title("Prédictions sans 'ENERGYSTARScore'")
plt.xlabel('Valeurs réelles')
plt.ylabel('Prédictions')

plt.tight_layout()
plt.show()

In [ ]:
# Visualisation des importances des caractéristiques pour le modèle Ridge
ridge_coefficients = model_with_feature.coef_

feature_names = X_train_with.columns

importances = np.abs(ridge_coefficients)
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
plt.title("Feature importances (Ridge Coefficients)")
plt.bar(range(X_train_with.shape[1]), importances[indices], color="r", align="center")
plt.xticks(range(X_train_with.shape[1]), [feature_names[i] for i in indices], rotation=90)
plt.xlim([-1, X_train_with.shape[1]])
plt.show()

**En conclusion :**  l'intégration de l'ENERGYSTAR Score dans notre modèle de prédiction de la consommation énergétique a démontré une valeur ajoutée significative.   

Il faut noter que l'ENERGYSTAR Score **n'est pas le facteur le plus influent dans notre modèle**. Bien qu'il contribue à l'amélioration des prédictions, d'autres caractéristiques, telles que la surface totale de la propriété et le type de propriété, jouent un rôle plus déterminant dans la consommation énergétique.

Cette constatation souligne **l'importance d'une analyse complète des caractéristiques** pour comprendre leur contribution relative à la performance énergétique. Elle met également en évidence la nécessité d'une approche multidimensionnelle dans la modélisation, où une combinaison de variables, plutôt qu'un seul indicateur, est utilisée pour obtenir des prédictions précises et fiables.